In [ ]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime, date
import os, time
import pickle
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, startstopFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Text, Select, Tab, Layout, VBox, HBox, Label, HTML, interact, interact_manual, interactive, IntSlider, Output

from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook(hide_banner=True)

# login to myplant()
cred()
mp = MyPlant(3600)
Engine._list_cached_validations();
#mp._fetch_installed_base(); # refresh local installed fleet database

#### Detailed Analysis of Engine Operation (c)2022 Dieter Chvatal

In [ ]:
# global constants
hh = '300px' # window height

# initialize global variables
fleet = None
e = None
fsm = None
rdf = pd.DataFrame([])

def do_lookup(lookup):
    def sfun(x):
        #return all([ (lookup in str(x['Design Number'])),  (x['OperationalCondition'] != 'Decommissioned') ])
        return (
            (str(lookup) in str(x['IB Site Name'])) or \
            (str(lookup) in str(x['Engine Type'])) or \
            (str(lookup) in str(x['Engine Version'])) or \
            (str(lookup) == str(x['Design Number'])) or \
            (str(lookup) == str(x['serialNumber'])) or \
            (str(lookup) == str(x['id']))) and \
            (x['OperationalCondition'] != 'Decommissioned')

    global fleet
    fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
    fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
    ddl = [f"{x['serialNumber']}  J{x['Engine Type']} {x['Engine Version']:<4} {x['Engine ID']} {x['IB Site Name']}" for i, x in fleet.iterrows()]
    ddl = [m for m in ddl]
    return ddl

In [ ]:
#########################################
# tab1
#########################################
tab1_out = widgets.Output()
query_list = []
qfn = '/opt/notebooks/assets/engines.pkl'

if os.path.exists(qfn):
    with open(qfn, 'rb') as handle:
        query_list = pickle.load(handle)
else:  
    query_list = [
        'Forsa Hartmoor',
        'BMW Landshut']

tdd = widgets.Combobox(
    value='',
    # placeholder='Choose Someone',
    options=query_list,
    description='Site Name:',
    #ensure_option=True,
    disabled=False,
    layout=widgets.Layout(width='600px'))
tw = Text(
    value='Forsa Hartmoor', 
    description='Site Name:', 
    disabled=False,
    layout=widgets.Layout(width='600px'))
es = Select(
    options=['-'], 
    value='-', 
    rows=10, 
    description='Engine:', 
    disabled=False, 
    layout=widgets.Layout(width='600px'))
sel = Text(
    value='-', 
    description='selected:', 
    disabled=True, 
    layout=widgets.Layout(width='400px'))
selno = Text(
    value='-', 
    description='Motor No:', 
    disabled=True, 
    layout=widgets.Layout(width='200px'))

def do_sel(*args):
    sel.value = es.value
    selno.value = str(list(es.options).index(es.value))

es.observe(do_sel, 'value')

@tab1_out.capture(clear_output=True)
def sbcb(but):
    elst = do_lookup(tdd.value)
    es.options = elst
    es.value = elst[0]
    if not tdd.value in query_list:
        query_list.append(tdd.value)
    #print(tdd.value, query_list)
    if os.path.exists(qfn):
        os.remove(qfn)
    with open(qfn, 'wb') as handle:
        pickle.dump(query_list, handle, protocol=5)
        
sb = Button(description='Lookup',disabled=False, button_style='primary')
sb.on_click(sbcb)
tab1 = VBox([
        HBox([tdd,sb]),
        es,
        HBox([sel,selno]),
        tab1_out
    ],
    layout=widgets.Layout(height=hh)
)

In [ ]:
#########################################
# tab2
#########################################
tab2_out = widgets.Output()
el = Text(
    value='-', description='selected:', disabled=True, 
    layout=widgets.Layout(width='603px')
)
t1 = widgets.DatePicker( 
    value=pd.to_datetime('2022-01-01'), 
    description='From: ',disabled=False
)
t2 = widgets.DatePicker( 
    value = date.today(), 
    description='To:',disabled=False
)
trun = widgets.Button(
    description='Run FSM',
    disabled=False, 
    button_style='primary'
)
tload = widgets.Button(
    description='load Results',
    disabled=False, 
    button_style='primary'
)

with open('./assets/Misterious_mist.gif', 'rb') as f:
    img = f.read()    
loading_bar = widgets.Image(
    value=img
)

#@tab2_out.capture(clear_output=True)
def fsm_run(b):
    motor = fleet.iloc[int(selno.value)]
    with tab2_out:
        tab2_out.clear_output()
        print()
        global fsm
        fsm = FSMOperator(e, p_from=t1.value, p_to=t2.value)
        tload.disable = fsm.exists
        fsm.run0(enforce=True, silent=False, debug=False)
        fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
        fsm.run2(silent = False)
        fsm.store()
        global rdf
        rdf = fsm.starts
        print()
        print(f"Starts: {rdf.shape[0]}") 
        print(f"Successful: {rdf[rdf['success'] == 'success'].shape[0]}, Failed: {rdf[rdf['success'] == 'failed'].shape[0]}, Undefined: {rdf[rdf['success'] == 'undefined'].shape[0]}")
        print(f"Starting reliability raw: {rdf[rdf['success'] == 'success'].shape[0]/(rdf.shape[0])*100.0:3.1f}% ")
        print(f"Starting reliability: {rdf[rdf['success'] == 'success'].shape[0]/(rdf.shape[0]-rdf[rdf['success'] == 'undefined'].shape[0])*100.0:3.1f}% ")
        pd.DataFrame.from_dict(e.dash, orient='index').T
trun.on_click(fsm_run)

def fsm_load(b):
    fsm.restore()
tload.on_click(fsm_load)

tab2 = VBox([HBox([el,trun]),HBox([t1,t2,tload]),tab2_out]) #,layout=widgets.Layout(height=hh)

In [ ]:
#########################################
# tab3
#########################################
tab3_out = widgets.Output()
mo = widgets.SelectMultiple( options=['undefined','OFF','MAN','AUTO'], value=['undefined','OFF','MAN','AUTO'], rows=4, description='modes: ', disabled=False)
succ = widgets.SelectMultiple( options=['success','failed','undefined'], value=['success','failed','undefined'], rows=3, description='success: ', disabled=False)

@tab3_out.capture(clear_output=True)
def show_overview(b):
    global rda
    rda = rdf[:].reset_index(drop='index')
    rda = rda[(rda['mode'].isin(mo.value) & rda['success'].isin(succ.value))].reset_index(drop='index')
    global rdb
    rdb = rda
    rde = rda
    rde['datetime'] = pd.to_datetime(rde['starttime'])
    sdict ={'success':1, 'failed':0, 'undefined':0.5}
    rde['isuccess'] = rde.apply(lambda x: sdict[x['success']], axis=1)
    vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout','isuccess']
    display(rde[vec].describe().round(2))
    dfigsize = (20,10)
    dset = [
        {'col':['cumstarttime'],'_ylim':(-600,800), 'color':'darkblue'},
        {'col':['runout'],'_ylim':(0,100) },
        {'col':['targetload'],'_ylim':(-4000,26000) },
        {'col':['ramprate'],'_ylim':(-5,7)},
        {'col':['loadramp'],'_ylim':(-150,900), 'color':'red'},
        {'col':['speedup'],'_ylim':(-100,200), 'color':'orange'},
        {'col':['synchronize'],'_ylim':(-20,400)},
        {'col':['startpreparation'],'_ylim':(-1000,800)},
        {'col':['count_warnings','count_alarms','isuccess'],'_ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,128,0,0.2)'] },
        {'col':['no'],'_ylim':(0,1000), 'color':['rgba(0,0,0,0.1)'] },
        #{'col':['count_warnings','count_alarms','no'],'ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,0,0,0.1)'] }
        ]
    dset = equal_adjust(dset, rde, do_not_adjust=[-1])
    ftitle = f"{fsm._e}"
    fig = dbokeh_chart(rde, dset, style='both', figsize=dfigsize ,title=ftitle);
    bokeh_show(fig)

t3_button = widgets.Button(description='Overview',disabled=False, button_style='primary')
t3_button.on_click(show_overview)
tab3 = VBox([HBox([el,t3_button]), HBox([mo,succ]),tab3_out])

In [ ]:
#########################################
# tab4
#########################################
tab4_out = widgets.Output()
pfigsize=(18,10)

def myfigures(e):
    return {
    'actors' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Ignition_ITPAvg'],'ylim': [-10, 30], 'color':'rgba(255,0,255,0.4)', 'unit':'°KW'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['Various_Values_PosThrottle','Various_Values_PosTurboBypass'],'ylim': [-10, 110], 'color':['rgba(105,105,105,0.6)','rgba(165,42,42,0.4)'], 'unit':'%'},
        ],
        'tecjet' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['TecJet_GasPress1'],'_ylim': [0, 3], 'color':'rgba(255,0,0,0.4)', 'unit':'mbar'},
        {'col':['TecJet_GasTemp1'],'_ylim': [0, 3], 'color':'rgba(255,0,255,0.4)', 'unit':'°C'},
        {'col':['TecJet_GasDiffPress'],'_ylim': [0, 3], 'color':'rgba(0,255,0,0.4)', 'unit':'mbar'},
        ],
        'lubrication' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
        {'col':['Hyd_PressOilDif'],'ylim': [0, 3], 'color':'black', 'unit': 'bar'},
        {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown', 'unit': 'bar'},
        {'col':['Hyd_TempOil','Hyd_TempCoolWat','Hyd_TempWatRetCoolOut'],'ylim': [0, 110], 'color':['#2171b5','orangered','hotpink'], 'unit':'°C'},
        ],
        'exhaust' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':e.dataItemsCyl('Exhaust_TempCyl*'),'ylim': [400, 700], 'unit':'°C'},
        {'col':e.dataItemsCyl('Knock_Valve_Noise_Cyl*'),'ylim': [0, 12000], 'unit':'mV'},
        ],
        'ignition' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':e.dataItemsCyl('Monic_VoltCyl*'),'ylim': [0, 100], 'unit':'kV'},
        {'col':e.dataItemsCyl('Ignition_ITPCyl*'),'ylim': [0, 40], 'unit':'°KW'},
        {'col':e.dataItemsCyl('Knock_KLS98_IntKnock_Cyl*'),'ylim': [-30, 60], 'unit':'%'},
        ],    
    }

def update_fig(x=0):
    global vv;
    rdbs = rdb[rdb.no == x]
    if not rdbs.empty:
        startversuch = rdbs.iloc[0]
        #startversuch = rdb.iloc[x]
        vv = startversuch.no; 
        #ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')} CumStart: {startversuch['cumstarttime']:0.1f}"
        #display(HTML(ftitle));
        #disp_result(startversuch)

        # PLotter
        lfigures = myfigures(fsm._e)
        plotdef, vset = cplotdef(mp, lfigures)
        #dmaxlength = 1800
        dmaxlength = None
        dminlength = None
        #startversuch = rdb.iloc[vv]
        ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
        data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=vset)
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        fig_handles = []
        for doplot in plotdef:
            dset = lfigures[doplot]
            ltitle = f"{ftitle} | {doplot}"
            if count_columns(dset) > 12: # no legend, if too many lines.
                fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, legend=False, figsize=pfigsize)
            else:
                fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, figsize=pfigsize)

            fig = FSM_add_Notations(fig, fsm, startversuch)
            disp_alarms(startversuch)
            disp_warnings(startversuch)
            fig = FSM_add_Alarms(fig, fsm, startversuch)
            fig = FSM_add_Warnings(fig, fsm, startversuch)
            fig_handles.append(bokeh_show(fig, notebook_handle=True))
        for h in fig_handles:
            push_notebook(handle=h)
    else:
        print(f"Start No {x} is not in the filtered results. please consider changing filters, if you want to access it.")

@tab4_out.capture(clear_output=True)
def cb(but):
    update_fig(_a.value)

def html_summary(*args):
    if not rdf.empty:
        sv = rdf.iloc[_a.value]
        global vv; vv = _a.value
        s = '''
        <style>
            table, td {
                border: 1px solid grey;
                border-collapse: collapse;
                padding: 0px 5px;
                text-align: right;
            }
            thead {
                text-align: left;
            }
        </style>
        '''
        summary = pd.DataFrame(sv[startstopFSM.run2filter_content]).T
        r = f"{s}{summary.to_html(escape=False, index=False)}"
        _e.value = r
    
vv = 0 
_a = widgets.IntText(description='No: ',layout=widgets.Layout(width='10%'))
_a.observe(html_summary, 'value')
_b = widgets.IntSlider(0, 0, max(rdf.shape[0]-1,1) , 1, layout=widgets.Layout(width='70%'))
mylink = widgets.jslink((_a, 'value'), (_b, 'value'))
_d = widgets.Button(description='Show Plots',disabled=False, button_style='primary')
_d.on_click(cb)
_e = widgets.HTML()
html_summary()
tab4 = widgets.VBox([el,widgets.HBox([_b, _a, _d]),_e, tab4_out]);

In [ ]:
#########################################
# tabs
#########################################
tabs_out = widgets.Output()
tabs = widgets.Tab()
tabs.children = [tab1, tab2, tab3, tab4]
tabs.set_title(0,'1. select Engine')
tabs.set_title(1,'2. Run FSMs')
tabs.set_title(2,'3. Filter Results')
tabs.set_title(3,'4. Start Plots')

@tabs_out.capture(clear_output=False)
def tabs_cb(*args):
    #tabs_out.clear_output()
    if tabs.selected_index > 0:
        if selno.value == '-':
            tabs_out.clear_output()
            print('Please select an Engine first.')
            tabs.selected_index = 0
        else:
            tabs_out.clear_output()
            print('Please Wait, Contacting MyPlant ...')
            display(loading_bar)
            global e
            e=Engine.from_fleet(mp, fleet.iloc[int(selno.value)])
            tabs_out.clear_output()            
            #print(e.dash)
            el.value = sel.value
            t1.value = pd.to_datetime(e['Commissioning Date'])
        if selno.value != '-' and ((fsm is None) or (len(fsm.starts) == 0)):
            tabs_out.clear_output()
            print('no Statemachine Results available, please run Statemachine or load results.')
            tabs.selected_index = 1
        if not rdf.empty:
            if _b.max < len(rdf):
                _b.max = rdf.shape[0]-1

tabs.observe(tabs_cb, 'selected_index')
display(VBox([tabs,tabs_out]))

In [ ]:
#update_fig(vv)

In [ ]:
if not rdf.empty:
    startversuch = rdf.iloc[vv]
    print("messages leading to state Switches:")
    print("-----------------------------------")
    for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=True)):
        print(f"{i:3} {v}")
    print(f"\nall messages during start attempt No.:{vv:4d} leading to state Switches:")
    print("---------------------------------------------------------------------")
    for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
        print(f"{i:3} {v}")

In [ ]:
if e is not None:
    mfn = e._fname + '_messages.txt'
    #fsm.save_messages(mfn)
    print(mfn)